## Objectives
- In this project I will use webscraping to get bank information from wikipedia
- Once I've got all the necessary data, I'll do Extract, Transform, and Load on the data i've collected.

### Import Libraries
We are going to be using this several python libraries below:
- pandas
- requests
- BeautifulSoup
- html5lib

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib

## Extract Data Using Web Scraping
The wikipedia webpage https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks provides information about largest banks in the world by various parameters. Scrape the data from the table 'By market capitalization' and store into JSON file. 

### Webpage Contents
Gather the contents of the webpage in the text format using `requests` library and assign it to the variable `bank_data`

In [53]:
url = 'https://web.archive.org/web/20200318083015/https://en.wikipedia.org/wiki/List_of_largest_banks'
bank_data = requests.get(url).text

### Scraping Data
Using the contents we have gathered before and `BeautifulSoup`, load the data from the `By market capitalization` table into `pandas` dataframe.

In [54]:
soup = BeautifulSoup(bank_data, "html.parser")

In [ ]:
print(soup.prettify())

Load the data from the `By market capitalization` table into a pandas dataframe. Using the empty dataframe `data` and the given loop extract the necessary data from each row and append it to the empty dataframe.

In [55]:
data = pd.DataFrame(columns=["Rank", "Bank Name", "Market Cap (US$ Billion)"])

for row in soup.find_all('tbody')[2].find_all('tr'):
    col = row.find_all('td')
    if len(col) == 0:
        continue
    else:
        data = data.append({"Rank": col[0].text.strip(), "Bank Name": col[1].text.strip(), "Market Cap (US$ Billion)": col[2].text.strip()}, ignore_index=True)

In [56]:
data.head()

,Rank,Bank Name,Market Cap (US$ Billion)
0,1,JPMorgan Chase,390.934
1,2,Industrial and Commercial Bank of China,345.214
2,3,Bank of America,325.331
3,4,Wells Fargo,308.013
4,5,China Construction Bank,257.399


### Loading the Data
Load the `pandas` dataframe created above into a JSON named `bank_market_cap.json` using the `to_json()` function.

In [14]:
data.to_json("bank_market_cap_USD.json")

## ETL
- Read JSON file type
- Extract data
- Transform data
- Save the transformed data

In [4]:
import glob
from datetime import datetime

### Set Paths

In [57]:
tmpfile = "tmpfile.tmp"                     # file used to store all extracted data
logfile = "logfile.txt"                     # all event logs will be stored in this file
targetfile = "transformed_data.csv"         # file where transformed data is stored

### Extract

In [58]:
def extract_json_data(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

#### Extract Function

In [59]:
def extract():
    extracted_data = pd.DataFrame(columns=["Rank", "Bank Name", "Market Cap (US$ Billion)"]) #create an empty dataframe to hold extracted_data

    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_json_data(jsonfile), ignore_index=True)

    return extracted_data

### Transform
- Changes the Market Cap (US$ Billion) column from USD to EUR
- Rounds the Market Cap (EUR€ Billion)` column to 3 decimal places
- Rename Market Cap (US$ Billion) to Market Cap (EUR€ Billion)

In [68]:
def transform(data1):
    data1['Market Cap (EUR€ Billion)'] = data1['Market Cap (US$ Billion)']
    data1['Market Cap (EUR€ Billion)'] = round(data1['Market Cap (US$ Billion)'] * 0.909895, 3)
    data1.drop('Market Cap (US$ Billion)', axis=1, inplace=True)
    return data1

### Loading

In [61]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

### Logging

In [62]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' #Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() #get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt", "a") as f:
        f.write(timestamp + ',' + message + '\n')

### Running ETL Process

In [69]:
log("ETL Job Started")

In [70]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,Rank,Bank Name,Market Cap (US$ Billion)
0,1,JPMorgan Chase,390.934
1,2,Industrial and Commercial Bank of China,345.214
2,3,Bank of America,325.331
3,4,Wells Fargo,308.013
4,5,China Construction Bank,257.399
...,...,...,...
65,66,Ping An Bank,37.993
66,67,Standard Chartered,37.319
67,68,United Overseas Bank,35.128
68,69,QNB Group,33.560


In [71]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,Rank,Bank Name,Market Cap (EUR€ Billion)
0,1,JPMorgan Chase,355.709
1,2,Industrial and Commercial Bank of China,314.108
2,3,Bank of America,296.017
3,4,Wells Fargo,280.259
4,5,China Construction Bank,234.206
...,...,...,...
65,66,Ping An Bank,34.570
66,67,Standard Chartered,33.956
67,68,United Overseas Bank,31.963
68,69,QNB Group,30.536


In [73]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [74]:
log("ETL Job Ended")